https://gis.stackexchange.com/questions/15135/using-field-to-rgb-mapping-for-symbology-in-qgis

In [75]:
%pylab inline
import logging as log
import os
import attr
log.getLogger().setLevel(log.INFO)

Populating the interactive namespace from numpy and matplotlib


In [76]:
# infolder = "/data/planmap/Planmap/DATA/mercury/PM-MER-MS-H05_3cc_01/".strip()
infolder = "/run/media/luca/data/planmap/Planmap/DATA/mercury/PM-MER-MS-H02_3cc_01/".strip()
outfolder = "/data/valentina_package/"

# projectname = PM-MER-MS-H05_3cc_01 # override here if needed
projectname = os.path.basename(os.path.normpath(infolder))
outfname = projectname +".gpkg"
outfullpath = os.path.join(outfolder, outfname)

log.info(f"working on project {projectname}, output will be in {outfolder}")

    

vectors_exts = ["shp"] # add more if needed


INFO:root:working on project PM-MER-MS-H02_3cc_01, output will be in /data/valentina_package/


In [77]:
import glob
def find_types(extensions, folder):
    found = []
    for vtype in extensions:
        found.extend(glob.glob(folder + f"/**/*.{vtype}", recursive = True))
        
    return found

In [78]:
shapefiles = find_types(vectors_exts, infolder)


import regex
def remove_filter_matches(fnames, filters=[".*5_classes.*"]):
    out = []
    for fname in fnames:
        for filter in filters:
            if not regex.match(filter, fname):
                out.append(fname)
            else:
                log.warning(f"removed from list {fname} due to filter {filter}")
                
    return out
    
shapefiles = remove_filter_matches(shapefiles)

In [79]:
shapefiles

['/run/media/luca/data/planmap/Planmap/DATA/mercury/PM-MER-MS-H02_3cc_01/vector/Mercury_H02_Galluzzi2016_JOM_shapefiles/Shapefiles/H02_Geologic_Contacts.shp',
 '/run/media/luca/data/planmap/Planmap/DATA/mercury/PM-MER-MS-H02_3cc_01/vector/Mercury_H02_Galluzzi2016_JOM_shapefiles/Shapefiles/H02_Surface_Features.shp',
 '/run/media/luca/data/planmap/Planmap/DATA/mercury/PM-MER-MS-H02_3cc_01/vector/Mercury_H02_Galluzzi2016_JOM_shapefiles/Shapefiles/H02_Linear_Features.shp',
 '/run/media/luca/data/planmap/Planmap/DATA/mercury/PM-MER-MS-H02_3cc_01/vector/Mercury_H02_Galluzzi2016_JOM_shapefiles/Shapefiles/H02_Geologic_Units.shp',
 '/run/media/luca/data/planmap/Planmap/DATA/mercury/PM-MER-MS-H02_3cc_01/vector/Mercury_H02_Galluzzi2016_JOM_shapefiles/Shapefiles/H02_Map_Boundary.shp']

In [80]:
log.info(f"found {len(shapefiles)} vectors")
for v in shapefiles:
    log.info(f"--> {v}")


INFO:root:found 5 vectors
INFO:root:--> /run/media/luca/data/planmap/Planmap/DATA/mercury/PM-MER-MS-H02_3cc_01/vector/Mercury_H02_Galluzzi2016_JOM_shapefiles/Shapefiles/H02_Geologic_Contacts.shp
INFO:root:--> /run/media/luca/data/planmap/Planmap/DATA/mercury/PM-MER-MS-H02_3cc_01/vector/Mercury_H02_Galluzzi2016_JOM_shapefiles/Shapefiles/H02_Surface_Features.shp
INFO:root:--> /run/media/luca/data/planmap/Planmap/DATA/mercury/PM-MER-MS-H02_3cc_01/vector/Mercury_H02_Galluzzi2016_JOM_shapefiles/Shapefiles/H02_Linear_Features.shp
INFO:root:--> /run/media/luca/data/planmap/Planmap/DATA/mercury/PM-MER-MS-H02_3cc_01/vector/Mercury_H02_Galluzzi2016_JOM_shapefiles/Shapefiles/H02_Geologic_Units.shp
INFO:root:--> /run/media/luca/data/planmap/Planmap/DATA/mercury/PM-MER-MS-H02_3cc_01/vector/Mercury_H02_Galluzzi2016_JOM_shapefiles/Shapefiles/H02_Map_Boundary.shp


In [81]:
def get_required_output_dirs(outfolder):
    outdirs = [outfolder]
    if len(shapefiles) > 0:
        outdirs.append(os.path.join(outfolder, "vector"))

        
    return outdirs
        
def create_paths(folders):
    for f in folders:
        os.makedirs(f, exist_ok=True)
        

In [82]:
flds = get_required_output_dirs(outfolder)
create_paths(flds)

In [83]:
# we now unify in a single geopackage all the data
outgpkg = os.path.join(outfolder, "vector", outfname)
log.info(outgpkg)

INFO:root:/data/valentina_package/vector/PM-MER-MS-H02_3cc_01.gpkg


In [84]:
def prepare_layer_name(fullname): # might want to personalize this
    lname = os.path.basename(fullname)[:-4] # strip estension from basename
    lname = lname[4:] # strip beginning
    lname = lname.replace("_", " ")
    lname = lname.title()
#     lname = lname.replace(" ", "_")
    
#     lname = ''.join(x for x in lname.title() if x != "_")
    return lname

In [85]:
import geopandas as gp
# first we load the data

tosave = {}
for vector in shapefiles:
    lname =prepare_layer_name(vector)
    log.info(f"{vector},\n   --> layer_name : {lname}")
    table = gp.GeoDataFrame.from_file(vector)
    tosave[lname] = table




    

INFO:root:/run/media/luca/data/planmap/Planmap/DATA/mercury/PM-MER-MS-H02_3cc_01/vector/Mercury_H02_Galluzzi2016_JOM_shapefiles/Shapefiles/H02_Geologic_Contacts.shp,
   --> layer_name : Geologic Contacts
INFO:fiona.ogrext:Failed to auto identify EPSG: 7
INFO:root:/run/media/luca/data/planmap/Planmap/DATA/mercury/PM-MER-MS-H02_3cc_01/vector/Mercury_H02_Galluzzi2016_JOM_shapefiles/Shapefiles/H02_Surface_Features.shp,
   --> layer_name : Surface Features
INFO:fiona.ogrext:Failed to auto identify EPSG: 7
INFO:root:/run/media/luca/data/planmap/Planmap/DATA/mercury/PM-MER-MS-H02_3cc_01/vector/Mercury_H02_Galluzzi2016_JOM_shapefiles/Shapefiles/H02_Linear_Features.shp,
   --> layer_name : Linear Features
INFO:fiona.ogrext:Failed to auto identify EPSG: 7
INFO:root:/run/media/luca/data/planmap/Planmap/DATA/mercury/PM-MER-MS-H02_3cc_01/vector/Mercury_H02_Galluzzi2016_JOM_shapefiles/Shapefiles/H02_Geologic_Units.shp,
   --> layer_name : Geologic Units
INFO:fiona.ogrext:Failed to auto identify EPSG

In [86]:
types = []

for lanme, t in tosave.items():
    types.append( t.geom_type[0] ) # we assume all features will be of similar type
log.info(types)   



INFO:root:['LineString', 'Polygon', 'LineString', 'Polygon', 'Polygon']


In [87]:
sort_pref = {"Point": 0, "MultiPoint":0, "LineString":10, "Polygon": 20}
scores = [sort_pref[key] for key in types]
scores


sorting = np.argsort(scores)
sorting

array([0, 2, 1, 3, 4])

In [88]:
alldata = list(tosave.items())

for id in sorting:
    lname = alldata[id][0]
    table = alldata[id][1]
    log.info(f"saving {lname}")
    table.to_file(outgpkg, driver="GPKG", layer=lname)

INFO:root:saving Geologic Contacts
INFO:root:saving Linear Features
INFO:root:saving Surface Features
INFO:root:saving Geologic Units
INFO:root:saving Map Boundary


# \todo add checks on the total number of layers received as input to the actual geopackage

In [89]:
tabfile = "/run/media/luca/data/planmap/Planmap/DATA/mercury/PM-MER-MS-H02_3cc_01/vector/Mercury_H02_Galluzzi2016_JOM_shapefiles/Symbology/codes.csv"
layername = "Geologic Units"
table = gp.GeoDataFrame.from_file( outgpkg, layer=layername)
table["UNIT"] = table["UNIT"].str.strip()
table["UNIT"] = table["UNIT"].str.lower()


INFO:fiona.ogrext:Failed to auto identify EPSG: 7


In [90]:
import pandas as pd
newtab = pd.DataFrame.from_csv(tabfile, sep=",", index_col=None )
newtab

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  


,label,UNIT,r,g,b
0,Smooth Plains,spp,255,190,190
1,Northern Smooth Plains,spn,245,162,122
2,Intermediate Plains,imp,245,122,122
3,Intercrater Plains,icp,137,90,68
4,Crater material-well preserved,c3,255,255,115
5,Crater material-degraded,c2,92,137,68
6,Crater material-heavily degraded,c1,115,0,0
7,Crater floor material-smooth,scf,255,255,175
8,Crater floor material-hummocky,hcf,205,170,102


In [91]:
tomatch

array(['c1', 'c2', 'c3', 'hcf', 'icp', 'imp', 'scf', 'spn', 'spp'],
      dtype=object)

In [92]:
matchname = "UNIT"

tomatch = np.unique(table[matchname])
testmatch = list(newtab[matchname])

for m in tomatch:
    if m not in testmatch:
        print(f"---> Check {m} -> not in list")
    else:
        print(f"{m} is ok")

c1 is ok
c2 is ok
c3 is ok
hcf is ok
icp is ok
imp is ok
scf is ok
spn is ok
spp is ok


In [93]:
fulldata = pd.merge(   table, newtab, on="UNIT")
fulldata
# fulldata = geopandas.GeoDataFrame(fulldata)
# fulldata.drop(columns=["Join_Count", "TARGET_FID"])

fulldata.to_file(outgpkg, layer=layername, driver="GPKG")
